<a href="https://colab.research.google.com/github/tcbic/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

**1) Load and clean the data (or determine the best method to drop observations when running tests).**

In [0]:
import pandas as pd
import numpy as np

In [0]:
#Create a DataFrame with appropriate column names.

voting_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', names= ['party'
                     , 'handicapped-infants'
                     , 'water-project-cost-sharing'
                     , 'adoption-of-the-budget-resolution'
                     , 'physician-fee-freeze'
                     , 'el-salvador-aid'
                     , 'religious-groups-in-schools'
                     , 'anti-satellite-test-ban'
                     , 'aid-to-nicaraguan-contras'
                     , 'mx-missile'
                     , 'immigration'
                     , 'synfuels-corporation-cutback'
                     , 'education-spending'
                     , 'superfund-right-to-sue'
                     , 'crime'
                     , 'duty-free-exports'
                     , 'export-administration-act-south-africa'])

In [22]:
#Let's take a look at our DataFrame.

voting_df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [23]:
voting_df.shape

(435, 17)

In [24]:
#Replace '?' in DataFrame with NaN.

voting_df = voting_df.replace('?', np.nan)

voting_df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [25]:
#All binary variables should be in the form of 0's or 1's instead of n's or y's.
#n=0 and y=1

voting_df = voting_df.replace({'n':0, 'y':1})

voting_df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


Example from class lecture:

In [0]:
#DataFrame for republicans

condition_r = (voting_df['party'] == 'republican')

republicans = voting_df[condition_r]

In [0]:
#DataFrame for democrats

condition_d = (voting_df['party'] == 'democrat')

democrats = voting_df[condition_d]

In [0]:
#For our purposes, we are going to treat each of these DataFrames as different samples.

**one sample t test**

In a one sample t test, we are testing the mean of one sample against **a null hypothesis of our choosing**.

The null hypothesis that we designate depends on how we have encoded our data and the kind of questions that we want to test.

If I have encoded votes as 0 for no and 1 for yes, I want to test Democratic support for an issue, and I use a null hypothesis of 0, then I am comparing Democrat voting support against a null hypothesis of no Democrat support at all for a given issue.

If I use a null hypothesis of .5, I am comparing the democrat voting support against a null hypothesis of democrats being neither in favor nor against a particular issue.

If I use a null hypothesis of 1, I am comparing the democrat voting support against a null hypothesis of all democrats being favor of a particular issue.

Lets use the 0 and .5 null-hypotheses to test the significance of those particular claims. They're all valid questions to be asking, they're just posing a slightly different question --testing something different.

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [46]:
#How many democrats voted yes or no regarding handicapped infants?

democrats['handicapped-infants'].value_counts()

1.0    156
0.0    102
Name: handicapped-infants, dtype: int64

In [47]:
#How many missing values are in the handicapped-infants column?

democrats['handicapped-infants'].isnull().sum()

9

In [0]:
#Comparing the mean of democrats voting for the handicapped-infants with a null hypothesis that the mean is 0.
#Remember: When you're doing this type of t test, you get to choose what you're hypothesis is!
#Note: nan_policy='omit' drops the missing values.

In [0]:
#Null hypothesis: There is 0 democrat support for this bill.
#Alternative hypothesis: There is non-0 democrat support for this bill.
#95% confidence level (.05 p-value threshold)

In [51]:
#The question our t test answers: Can we say that there is no democrat support for this bill?

ttest_1samp(democrats['handicapped-infants'], 0, nan_policy='omit')

Ttest_1sampResult(statistic=19.825711173357988, pvalue=1.0391992873567661e-53)

In [0]:
#In assessing the results of this t test, we reject the null hypothesis. We cannot say that there is 0 democrat support for this bill.

In [0]:
#We can ask a different question by changing the null hypothesis.
#For example, changing the null hypothesis to .5 is testing whether democrat support is split for handicapped-infants.
#Let's run this test.

#Ultimately, we are testing the validity of the null hypothesis.

In [52]:
ttest_1samp(democrats['handicapped-infants'], .5, nan_policy='omit')

Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372)

In [0]:
#The p-value isn't as extreme. We can see that democrat support on this issue is a lot closer to 50-50 than it is to 0 (look at value_counts above to see how democrats voted).
#We still reject this null hypothesis, and this proves to be statistically significant.

#More formal way of communicating results:
#Given the results of the above test, I would REJECT the null hypothesis that democrat support for the handicapped-infants bill is 
#50-50(split vote amongst dems) at the 95% confidence level.

**two sample t test (for means)**

In [53]:
#Take a look at the two means.

print(democrats['export-administration-act-south-africa'].mean())

print(republicans['export-administration-act-south-africa'].mean())

#We see that over 93% of democrats support this, and over 65% of republicans support this.
#So, we could claim that there is bi-partisan support.

0.9351351351351351
0.6575342465753424


In [57]:
#Let's run the test.

#Is the amount of democrat support the same as the amount of republican support?

#Null hypothesis: The means are the same (These two parties support this the same amount.).
#Alternative hypothesis: The means are not the same.

ttest_ind(democrats['export-administration-act-south-africa'], republicans['export-administration-act-south-africa'], nan_policy='omit')

Ttest_indResult(statistic=6.849454815841208, pvalue=3.652674361672226e-11)

In [0]:
#Given the results of the t test, we reject the null hypothesis. Therefore, the means are different.

In [0]:
#Note: As the t statistic goes up, the p-value goes down. They are inversely correlated.

**2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01.**

**3) Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01.**

**4) Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference).**

*   Null hypothesis: The means are the same.
*   Alternative hypothesis: The means are not the same.

In [80]:
#The mean of each respective party is representative of the amount of support in this case as it relates to water project cost sharing.

print(democrats['water-project-cost-sharing'].mean())
print(republicans['water-project-cost-sharing'].mean())

0.502092050209205
0.5067567567567568


In [81]:
ttest_ind(democrats['water-project-cost-sharing'], republicans['water-project-cost-sharing'], nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
#Given the results of this t test, we fail to reject the null hypothesis. This proves that there is a similar ammount of support amongst democrats and republicans.